# Continue writing functions to find the closest point on a route to our actual data point

Started by Nathaniel on Saturday, June 15, 2019

In [1]:
%load_ext autoreload
%autoreload 2

!date
!whoami

import numpy as np
import pandas as pd

Wed Jun 19 20:57:32 PDT 2019
ndbs


## Import my closest point module and read in a GTFS `shapes.txt` file

In [3]:
import find_closest_route_point as f

In [4]:
!ls ../data/source/gtfs_20180815/

agency.txt          calendar.txt        fare_rules.txt      stop_times.txt
block.txt           calendar_dates.txt  routes.txt          stops.txt
block_trip.txt      fare_attributes.txt shapes.txt          trips.txt


In [5]:
shapes_df = pd.read_csv('../data/source/gtfs_20180815/shapes.txt')
shapes_df.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,10002011,47.612137,-122.281769,1,0.0
1,10002011,47.612144,-122.281784,2,5.8
2,10002011,47.612148,-122.281830,3,13.5
3,10002011,47.612141,-122.281853,4,22.0
4,10002011,47.612102,-122.281921,5,45.0


# Testing, testing...

In [6]:
def get_two_things(x):
    return x+1, x-1

a = [1,2,3]
b = [get_two_things(x) for x in a]
b

[(2, 0), (3, 1), (4, 2)]

In [7]:
c = 3
c = -c
c

-3

## Test my "point data" functions

In [8]:
shape_id = 10002011
shape_pt_sequence = 6
point_data = f.get_shape_point_data(shapes_df, shape_id, shape_pt_sequence)
point_data

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
5,10002011,47.612129,-122.28199,6,64.2


In [11]:
adjacent_point_data = f.get_adjacent_shape_point_data(shapes_df, point_data.index[0])
adjacent_point_data

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
4,10002011,47.612102,-122.281921,5,45.0
6,10002011,47.612160,-122.282021,7,75.4


## Create a fake vehicle location to test with

In [13]:
1e-5

1e-05

In [26]:
point_coords = point_data[['shape_pt_lon','shape_pt_lat']].values
point_coords

array([[-122.28199  ,   47.6121292]])

In [23]:
veh_coords = np.random.normal(point_coords, 1e-4)
veh_coords

array([[-122.28201403,   47.61196255]])

In [24]:
adjacent_point_coords = adjacent_point_data[['shape_pt_lon','shape_pt_lat']].values
adjacent_point_coords

array([[-122.281921 ,   47.6121025],
       [-122.282021 ,   47.6121597]])

## Test point projection function with broadcasting over both adjacent points

Haha! I didn't initially design it to work on multiple line segments at once, but all I had to do was add `axis=1` to the distance calculation, because `numpy` is smart like that.

In [31]:
f.get_projection_and_dist_ratio(veh_coords, point_coords, adjacent_point_coords)

(array([[-122.28194578,   47.6121919 ],
        [-122.28200987,   47.61205757]]), array([[ 0.64093122, -2.34840677]]))

In [32]:
closest_point, dist_ratio = f.get_projection_and_dist_ratio(veh_coords, point_coords, adjacent_point_coords)

In [28]:
direction = adjacent_point_coords - point_coords
direction

array([[ 6.90e-05, -2.67e-05],
       [-3.10e-05,  3.05e-05]])

In [36]:
np.sum(direction**2, axis=1) # Check that axis=1 is correct for distance calculation

array([5.47389e-09, 1.89125e-09])

## Compute squared distance from vehicle to both projected points

In [41]:
veh_coords

array([[-122.28201403,   47.61196255]])

In [38]:
closest_point

array([[-122.28194578,   47.6121919 ],
       [-122.28200987,   47.61205757]])

In [40]:
veh_coords-closest_point

array([[-6.82517839e-05, -2.29356866e-04],
       [-4.15866155e-06, -9.50279982e-05]])

In [43]:
dist_squared = np.sum((veh_coords-closest_point)**2, axis=1)
dist_squared

array([5.72628778e-08, 9.04761492e-09])

## Get the point and distance ratio corresponding to the minimum distance

In [44]:
np.argmin(dist_squared)

1

In [45]:
dist_squared[1:]

array([9.04761492e-09])

In [46]:
np.argmin(dist_squared[1:])

0

In [48]:
min_index = np.argmin(dist_squared)
closest_point[min_index]

[-122.28200987   47.61205757]


In [49]:
dist_ratio # Wrong shape -> Index out of bounds if you do dist_ratio[min_index]

array([[ 0.64093122, -2.34840677]])

In [54]:
dist_ratio.reshape(2,)

array([ 0.64093122, -2.34840677])

In [55]:
dist_ratio.reshape(2,)[min_index]

-2.348406772138713

In [56]:
type(closest_point[min_index])

numpy.ndarray